<a href="https://colab.research.google.com/github/PintoPaola/Inteligencia-Artificial/blob/main/lab3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [258]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [259]:
# utilizado para la manipulación de directorios y rutas
import os

# Cálculo científico y vectorial para python
import numpy as np

# Libreria para graficos
from matplotlib import pyplot

# Modulo de optimizacion en scipy
from scipy import optimize
import pandas as pd

from scipy import optimize
# modulo para cargar archivos en formato MATLAB
# from scipy.io import loadmat

# le dice a matplotlib que incruste gráficos en el cuaderno
%matplotlib inline

Este código carga un conjunto de datos, lo prepara para su uso en el entrenamiento y la evaluación de modelos de aprendizaje automático y proporciona información básica sobre los datos, como el número de ejemplos de entrenamiento y las formas de las matrices de características y etiquetas.

In [260]:
# Tamaño de la capa de entrada
input_layer_size = 11

# Número de etiquetas
num_labels = 7

# Cargar el conjunto de datos desde el archivo CSV usando Pandas
data = pd.read_csv('/content/drive/MyDrive/IA/DATASET/covertype1.csv', header=None)

# Separar las características (X) y las etiquetas (y)
X = data.iloc[:, :-1]  # Todas las columnas excepto la última
y = data.iloc[:, -1]   # Última columna

# Ajustar las etiquetas para que vayan de 0 a 6 en lugar de 1 a 7
y[y == 7] = 0

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Número de ejemplos de entrenamiento
m = y_train.size
print("Número de ejemplos de entrenamiento:", m)
print("Forma de X:", X.shape)
print("Forma de y:", y.shape)


Número de ejemplos de entrenamiento: 16800
Forma de X: (21001, 11)
Forma de y: (21001,)


In [261]:
print("Primera fila de X:")
print(X_train_norm.iloc[0, :])

print("Valores de y:")
print(y)


Primera fila de X:
0    -0.077216
1     1.830114
2     1.725517
3    -1.065149
4    -0.779041
5     0.774472
6    -1.985480
7    -2.171749
8     0.222809
9     1.580234
10    1.089594
Name: 19400, dtype: float64
Valores de y:
0        5
1        5
2        2
3        2
4        5
        ..
20996    1
20997    1
20998    1
20999    1
21000    1
Name: 11, Length: 21001, dtype: int64


In [262]:
# Normalizar las características (opcional, dependiendo de tu aplicación)
def featureNormalize(X):
    X_norm = X.copy()
    mu = np.mean(X, axis=0)
    sigma = np.std(X, axis=0)
    X_norm = (X - mu) / sigma
    return X_norm, mu, sigma

In [263]:
# Llamar a la función de normalización con los datos de entrenamiento
X_train_norm, mu, sigma = featureNormalize(X_train)

In [264]:
# Configurar la matriz adecuadamente, y agregar una columna de unos que corresponde al termino de intercepción.
m, n = X.shape
X = X_train_norm

Sirve para mostrar algunas muestras aleatorias del conjunto de entrenamiento normalizado. Esto puede ser útil para visualizar cómo se ven algunas muestras después de la normalización o para verificar la distribución de las características.

In [265]:
# Selecciona aleatoriamente 100 puntos de datos para mostrar
rand_indices = np.random.choice(len(X_train_norm), 100, replace=False)
sel = X_train_norm.iloc[rand_indices, :]

# Imprimir algunas muestras seleccionadas aleatoriamente
print("Muestras seleccionadas aleatoriamente:")
print(sel)


Muestras seleccionadas aleatoriamente:
             0         1         2         3         4         5         6   \
12682 -1.223964  1.794378  1.249991  0.686866  2.602690 -0.683386 -1.708742   
18868  0.514205 -0.108610 -0.533234 -0.908252 -0.779041  2.263695  0.816496   
7574  -0.185505 -1.064571 -1.008760 -0.400952 -0.635138  0.040525  0.159242   
12251 -1.926451 -0.465979 -0.533234 -0.908252 -0.725077 -0.741098  0.851088   
2146   1.486025 -0.608926  0.180056 -0.845493 -0.671114  0.103883  0.954865   
...         ...       ...       ...       ...       ...       ...       ...   
366    0.914040 -0.787611  0.417819 -0.061009 -0.940933  0.756280  0.643534   
2660  -0.191058 -0.135413  1.249991 -0.500320 -0.113488 -1.097407  1.266195   
19280 -0.585339 -0.778676 -1.127642  0.064509 -0.239403 -0.908588  0.401388   
9881   0.317065 -0.930558 -0.414352 -0.751355 -0.473246 -1.081725  0.332203   
14347  1.694273  0.731206 -0.770997  0.603188 -0.023548  1.034804 -0.290458   

            

mapea el resultado de la regresión

In [266]:
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

calcula el costo y el gradiente de la regresión logística regularizada

In [267]:
def lrCostFunction(theta, X, y, lambda_):

    m = y.size

    # convierte las etiquetas a valores enteros si son booleanos
    if y.dtype == bool:
        y = y.astype(int)

    J = 0
    grad = np.zeros(theta.shape)

    h = sigmoid(X.dot(theta.T))

    temp = theta
    temp[0] = 0

    J = (1 / m) * np.sum(-y.dot(np.log(h)) - (1 - y).dot(np.log(1 - h))) + (lambda_ / (2 * m)) * np.sum(np.square(temp))

    grad = (1 / m) * (h - y).dot(X)
    grad = grad + (lambda_ / m) * temp

    return J, grad


 oneVsAll se utiliza para entrenar clasificadores de regresión logística uno contra todos.

In [268]:
# Entrenar clasificadores de regresión logística uno contra todos
def oneVsAll(X, y, num_labels, lambda_):
    m, n = X.shape
    all_theta = np.zeros((num_labels, n + 1))
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    for c in range(num_labels):
        initial_theta = np.zeros(n + 1)
        options = {'maxiter': 50}
        res = optimize.minimize(lrCostFunction,
                                initial_theta,
                                 (X, (y == c),
                                  lambda_),
                                jac=True,
                                method='CG',
                                options=options)
        all_theta[c] = res.x
    return all_theta


In [269]:
# Entrenar los clasificadores uno contra todos
lambda_ = 0.1
all_theta = oneVsAll(X_train_norm, y_train, num_labels, lambda_)

In [270]:
print(all_theta)

[[-10.01006814   7.13072022   0.05757404  -0.01930118  -0.36106877
   -0.03743146   0.10414494   0.17215406  -0.04374517  -0.51536345
    0.337743    -0.33922315]
 [ -2.43272241   1.35968695  -0.08383314  -0.04069454  -0.10380459
   -0.06864508  -0.24931545   1.53838125  -1.58136603   2.54573723
    0.24079413   0.45272377]
 [ -1.13977942  -0.29258803  -0.10936132   0.21852482   0.45179699
   -0.44596553   0.42709738   0.08719684   0.33947406   0.12578595
    0.82276637   0.74026603]
 [ -5.05412411  -0.78383949  -0.17952748   0.39670583   0.32426904
    0.09250392  -0.62726794  -0.08210772   0.32357719   0.13209386
   -0.49169133  -2.44664092]
 [ -8.57221812  -4.71187044   0.11537991  -0.48433758  -0.89496587
    0.57727164   1.27169829   0.23607552   1.30872452  -1.39235404
    1.67905301  -4.51687929]
 [ -2.42302284   0.38538486   0.08965043   0.24293973  -0.22886866
    0.15087666  -1.00995234   0.98658441  -0.12610554   0.63470128
   -0.52366182   0.55667281]
 [ -5.49115107  -0.739


Esta función predictOneVsAll se utiliza para predecir las etiquetas utilizando los clasificadores entrenados mediante el enfoque uno contra todos en un problema de clasificación multiclase utilizando regresión logístic

In [271]:
def predictOneVsAll(all_theta, X):

    m = X.shape[0]
    num_labels = all_theta.shape[0]

    p = np.zeros(m)

    # Agregar unos a la matriz de datos X
    X = np.concatenate([np.ones((m, 1)), X], axis=1)
    p = np.argmax(sigmoid(X.dot(all_theta.T)), axis=1)

    return p


In [272]:
print(X.shape)

# Predecir etiquetas para el conjunto de entrenamiento
pred_train = predictOneVsAll(all_theta, X_train_norm)
accuracy_train = np.mean(pred_train == y_train) * 100
print(f"Precisión del conjunto de entrenamiento: {accuracy_train:.2f}%")

# Predecir etiquetas para el conjunto de prueba
X_test_norm = (X_test - mu) / sigma
pred_test = predictOneVsAll(all_theta, X_test_norm)
accuracy_test = np.mean(pred_test == y_test) * 100
print(f"Precisión del conjunto de prueba: {accuracy_test:.2f}%")

# Imprimir las etiquetas correspondientes a un rango de filas seleccionadas
print(y[10680:21000])

# Seleccionar un rango de filas para el conjunto de prueba
XPrueba = X.iloc[10:10600, :].copy()
print(XPrueba.shape)

# Agregar la columna de unos al conjunto de datos de prueba
XPrueba = np.concatenate([np.ones((10590, 1)), XPrueba], axis=1)
print(XPrueba.shape)

# Predecir etiquetas para el conjunto de prueba seleccionado
p = np.argmax(sigmoid(XPrueba.dot(all_theta.T)), axis=1)
print(p)

# Imprimir las etiquetas correspondientes a un rango de filas seleccionadas
print(y[10:21000])



(16800, 11)
Precisión del conjunto de entrenamiento: 64.71%
Precisión del conjunto de prueba: 64.39%
10680    1
10681    1
10682    1
10683    6
10684    2
        ..
20995    1
20996    1
20997    1
20998    1
20999    1
Name: 11, Length: 10320, dtype: int64
(10590, 11)
(10590, 12)
[0 2 4 ... 0 2 2]
10       5
11       2
12       2
13       5
14       5
        ..
20995    1
20996    1
20997    1
20998    1
20999    1
Name: 11, Length: 20990, dtype: int64
